>
> # MaaSSim tutorial

 ### spatial patterns

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Polygon, Point, LineString
import pandas as pd
import geopandas as gpd
import folium
from IPython.display import Image
import selenium.webdriver
import shapely.geometry
import osmnx as ox
import pyproj
from geojson.feature import Feature, FeatureCollection
from shapely.geometry import Point, Polygon, LineString
phantom_path = '/Users/rkucharski/anaconda3/lib/node_modules/phantomjs-prebuilt/lib/phantom/bin/phantomjs'
import h3
import json

In [3]:
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
import seaborn as sns
import h3
import pandas as pd
import matplotlib.pyplot as plt
from dotmap import DotMap
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

prepare

In [4]:
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, collect_results  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate, simulate_parallel
from MaaSSim.traveller import travellerEvent
from MaaSSim.driver import driverEvent

In [5]:
params = get_config('../../data/config/delft.json')  # load configuration

In [6]:
params.paths.dumps = 'hexes'
params.times.patience = 1200
params.simTime = 4
params.parallel.nThread = 4
params.parallel.nReplications = 20
params.nV = 5
params.nP = 30
params.nD = 30

In [7]:
inData = load_G(inData, params)  # load network graph 

In [8]:
space = DotMap()
space.nP = [30]
space.nV = [5]
dict(space)

{'nP': [30], 'nV': [5]}

simulate

In [9]:
#simulate_parallel(params = params, search_space = space)
sim = simulate(params = params)


20
0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
Name: pax_id, dtype: int64
2156
2156
19-06-23 22:27:43-WARNING-Setting up 4h simulation at 2023-06-19 23:38:34 for 5 vehicles and 30 passengers in Delft, Netherlands
19-06-23 22:27:43-INFO--------------------	Starting simulation	-------------------
1 matched with 0.0
19-06-23 22:27:43-INFO-pax    0  got to dest at                           2023-06-19 23:49:54
3 matched with 1.0
5 matched with 2.0
19-06-23 22:27:43-INFO-pax    1  got to dest at                           2023-06-20 00:11:12
19-06-23 22:27:43-INFO-pax    2  got to dest at                           2023-06-20 00:14:47
4 matched with 3.0
3 matched with 4.0
19-06-23 22:27:43-INFO-pax    3  got to dest at                           2023-06-20 00:19:57
5 matched with 5.0
19-06-23 22:27:43-INFO-pax    4  got to dest at                           2023

In [10]:
KPI_RPOS = pd.DataFrame()
KPI_DRIVE = pd.DataFrame()
WAIT_MEAN = []
RPOS_MEAN = []
import numpy as np
#print(sim.res[0].pax_exp)
#print(sim.res[0].veh_exp)

clmname = []
for i in range (0,params.nD):
    print(sim.res[i].pax_exp['LOSES_PATIENCE'].sum()/1215)
    performance = sim.res[i].veh_exp
    pax = sim.res[i].pax_exp
    wait = pax['WAIT']
    repos = performance['REPOSITION']
    drive = performance['TRAVEL']
    clmname.append('Day '+str(i+1))
    WAIT_MEAN.append(int(wait.mean()))
    RPOS_MEAN.append(int(repos.mean()))
    KPI_RPOS = pd.concat([KPI_RPOS,repos],axis = 1)
    KPI_DRIVE = pd.concat([KPI_DRIVE,drive],axis = 1)

WAIT_MEAN = pd.DataFrame(WAIT_MEAN).T
RPOS_MEAN = pd.DataFrame(RPOS_MEAN).T
KPI_RPOS.columns = clmname
KPI_DRIVE.columns = clmname
WAIT_MEAN.columns = clmname
print(KPI_DRIVE)
print(KPI_RPOS)
print(RPOS_MEAN)
print(WAIT_MEAN)

#sim.res[0].veh_exp[['TRAVEL','WAIT','OPERATIONS','REPOSITION', 'PAX_KM', 'REVENUE']].sample(5)
#for i in range(0,params.nD):
#    print(sim.runs[i].queues)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
    Day 1   Day 2   Day 3   Day 4   Day 5   Day 6   Day 7   Day 8   Day 9  \
1  1113.0  1381.0  1129.0  1210.0  1164.0  1941.0  1175.0   604.0   879.0   
2   534.0  1143.0   885.0  1084.0   244.0  1621.0   508.0  1164.0  1126.0   
3  1647.0  1047.0   788.0   509.0  1355.0   241.0  1429.0  1375.0  1665.0   
4  1344.0  1157.0   778.0  1867.0  1151.0   550.0  1071.0  1311.0  1351.0   
5   876.0   786.0  1934.0   844.0  1600.0  1161.0  1331.0  1060.0   493.0   

   Day 10  ...  Day 16  Day 17  Day 18  Day 19  Day 20  Day 21  Day 22  \
1   855.0  ...   846.0  1095.0   869.0  1764.0  1657.0  1960.0  1818.0   
2   875.0  ...  1344.0  1445.0   564.0   244.0   244.0   244.0   244.0   
3   756.0  ...   519.0   830.0  1104.0  1694.0  1415.0  1694.0  1888.0   
4  1091.0  ...  1299.0   796.0  1110.0  1039.0  1114.0   830.0   778.0   
5  1937.0  ...  1506.0  1348.0  1867.0   773.0  1084.0   786.0   78

### collect

demand structure

In [11]:
res = collect_results('hexes')

BadZipFile: Bad CRC-32 for file 'passengers.csv'

In [ ]:
def plot_hex_wait_vehs(inData, res, APERTURE_SIZE = 9, threshold = 3, col = 'wait_time'):  
    traces = res.rides[['veh','pos','t']]
    diffs = traces.diff()
    diffs.columns = [_+"_diff" for _ in diffs.columns]
    traces = pd.concat([traces, diffs], axis = 1)
    trips = traces[(traces.veh_diff==0) & (traces.pos_diff==0)][['pos','t_diff']]
    trips.columns = ['pos','wait_time']
    
    trips['x'] = trips.apply(lambda row: inData.nodes.loc[row.pos].x, axis = 1)
    trips['y'] = trips.apply(lambda row: inData.nodes.loc[row.pos].y, axis = 1)
    trips['hex_o_{}'.format(APERTURE_SIZE)] = trips.apply(lambda row: h3.geo_to_h3(row.y,row.x,APERTURE_SIZE),axis = 1) 
    trips = pd.DataFrame(trips)
    
    
    col_geom = 'hex_o_{}'.format(APERTURE_SIZE)
    hexes = pd.Series(list(set(list(trips[col_geom].unique())+list(trips[col_geom].unique())))).to_frame(col_geom)
    hexes = hexes.set_index(col_geom)
    hexes[col_geom] = hexes.index.copy()
    hexes['nobs'] = trips.groupby(col_geom).size()
    hexes = hexes[hexes['nobs']>threshold]
    hexes['geom'] = hexes.apply(lambda x: {"type": "Polygon","coordinates": [h3.h3_to_geo_boundary(h = x[col_geom], geo_json = True)]}, axis = 1)
    aggr = trips.groupby(col_geom)[col]
    hexes['nobs'] = aggr.size()
    hexes[col] = aggr.mean()/60
    hexes[col+'_std'] = aggr.std()
    list_features = []
    for i, row in hexes.iterrows():
        feature = Feature(geometry = row["geom"],
                          id = row[col_geom],
                          properties = {"resolution": 9})
        list_features.append(feature)
    
    feat_collection = FeatureCollection(list_features)
    geojson_hexes = json.dumps(feat_collection)
    CENTER = list(inData.nodes.loc[inData.stats.center][['y','x']].values)
    tile = 'cartodbpositron'
    base_map = folium.Map(location=CENTER, zoom_start=13,tiles=tile, zoomControl =  False)
    bins = [0, 1, 2, 3, 5, 10, 15, 20, 25, 115]
    m = folium.Choropleth(geo_data = geojson_hexes,data = hexes, columns = [col_geom,col],key_on ="feature.id",
            fill_color='Blues', control = False, bins = bins,
            fill_opacity=0.7, line_opacity=0.1).add_to(base_map)
    return base_map
plot_hex_wait_vehs(inData, res)

In [ ]:
res = collect_results("hexes")
def plot_hex_map(inData, res, APERTURE_SIZE = 9, threshold = 3, col = 'wait_time'):
    res.requests['wait_time'] = res.requests.apply(lambda x: res.pax_exp[res.pax_exp['dt'] == x['dt']].loc[x.name].MEETS_DRIVER_AT_PICKUP, axis = 1)
    res.requests['x'] = res.requests.apply(lambda row: inData.nodes.loc[row.origin].x, axis = 1)
    res.requests['y'] = res.requests.apply(lambda row: inData.nodes.loc[row.origin].y, axis = 1)
    res.requests['hex_o_{}'.format(APERTURE_SIZE)] = res.requests.apply(lambda row: h3.geo_to_h3(row.y,row.x,APERTURE_SIZE),axis = 1) 
    trips = res.requests
    
    col_geom = 'hex_o_{}'.format(APERTURE_SIZE)
    hexes = pd.Series(list(set(list(trips[col_geom].unique())+list(trips[col_geom].unique())))).to_frame(col_geom)
    hexes = hexes.set_index(col_geom)
    hexes[col_geom] = hexes.index.copy()
    hexes['nobs'] = trips.groupby(col_geom).size()
    hexes = hexes[hexes['nobs']>threshold]
    hexes['geom'] = hexes.apply(lambda x: {"type": "Polygon","coordinates": [h3.h3_to_geo_boundary(h = x[col_geom], geo_json = True)]}, axis = 1)
    aggr = trips.groupby(col_geom)[col]
    hexes['nobs'] = aggr.size()
    hexes[col] = aggr.mean()/60
    hexes[col+'_std'] = aggr.std()
    list_features = []
    for i, row in hexes.iterrows():
        feature = Feature(geometry = row["geom"],
                          id = row[col_geom],
                          properties = {"resolution": 9})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)
    geojson_hexes = json.dumps(feat_collection)
    CENTER = list(inData.nodes.loc[inData.stats.center][['y','x']].values)
    tile = 'cartodbpositron'
    base_map = folium.Map(location=CENTER, zoom_start=13,tiles=tile, zoomControl =  False)
    bins = [0, 1, 2, 3, 5, 10, 15, 20, 25, 167]

    m = folium.Choropleth(geo_data = geojson_hexes,data = hexes, columns = [col_geom,col],key_on ="feature.id",
            fill_color='Blues', control = False, bins = bins,
            fill_opacity=0.7, line_opacity=0.1).add_to(base_map)
    return base_map
    
plot_hex_map(inData, res, APERTURE_SIZE = 9)

-------------
Rafal Kucharski, 2020, Delft, r.m.kucharski@tudelft.nl